<a href="https://colab.research.google.com/github/Ytlee413/Bioinfo/blob/main/HW11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries and Load Data

In [ ]:
# !pip install pandas pulp
# !pip install xlrd
# Import necessary libraries
import pandas as pd
import pulp

# Load the diet data from the provided Excel file
file_path = "diet.xls"
data = pd.read_excel(file_path)

# Display the first few rows to understand the data structure
display(data)

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0


### Organize the Data

In [ ]:
# Select the rows containing food items and their attributes
food_data = data.iloc[:len(data)-3, :]  # Remove last three rows that contain min/max constraints

# Select rows with minimum and maximum nutrient constraints
nutrient_constraints = data.iloc[len(data)-2:, 2:].reset_index(drop=True)

# Display the organized data to verify
display(food_data)
display(nutrient_constraints)


,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6


,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0
1,Maximum daily intake,2500.0,240.0,70.0,2000.0,450.0,250.0,100.0,10000.0,5000.0,1500.0,40.0


### Extract Costs and Nutrient Values

In [ ]:
# Extract costs and nutrient values into dictionaries for easier reference
food_costs = {row['Foods']: row['Price/ Serving'] for _, row in food_data.iterrows()}
calories = {row['Foods']: row['Calories'] for _, row in food_data.iterrows()}
proteins = {row['Foods']: row['Protein g'] for _, row in food_data.iterrows()}
fats = {row['Foods']: row['Total_Fat g'] for _, row in food_data.iterrows()}
sodium = {row['Foods']: row['Sodium mg'] for _, row in food_data.iterrows()}
carbohydrates = {row['Foods']: row['Carbohydrates g'] for _, row in food_data.iterrows()}
fiber = {row['Foods']: row['Dietary_Fiber g'] for _, row in food_data.iterrows()}

# Minimum and maximum constraints
min_intake = nutrient_constraints.iloc[0]
max_intake = nutrient_constraints.iloc[1]

### Define the Optimization Model

In [ ]:
# Define the linear programming problem
prob = pulp.LpProblem("Diet_Optimization", pulp.LpMinimize)

# Decision variables for each food item (quantity in the diet)
food_vars = {food: pulp.LpVariable(f"x_{food}", lowBound=0, cat='Continuous') for food in food_costs.keys()}

# Objective function: Minimize total cost
prob += pulp.lpSum([food_costs[food] * food_vars[food] for food in food_costs]), "Total Cost"

### Add Nutritional Constraints

In [ ]:
# Add constraints for each nutrient based on min and max intake

# Calories
prob += pulp.lpSum([calories[food] * food_vars[food] for food in calories]) >= min_intake['Calories'], "Min Calories"
prob += pulp.lpSum([calories[food] * food_vars[food] for food in calories]) <= max_intake['Calories'], "Max Calories"

# Proteins
prob += pulp.lpSum([proteins[food] * food_vars[food] for food in proteins]) >= min_intake['Protein g'], "Min Protein"
prob += pulp.lpSum([proteins[food] * food_vars[food] for food in proteins]) <= max_intake['Protein g'], "Max Protein"

# Fats
prob += pulp.lpSum([fats[food] * food_vars[food] for food in fats]) >= min_intake['Total_Fat g'], "Min Fat"
prob += pulp.lpSum([fats[food] * food_vars[food] for food in fats]) <= max_intake['Total_Fat g'], "Max Fat"

# Sodium
prob += pulp.lpSum([sodium[food] * food_vars[food] for food in sodium]) >= min_intake['Sodium mg'], "Min Sodium"
prob += pulp.lpSum([sodium[food] * food_vars[food] for food in sodium]) <= max_intake['Sodium mg'], "Max Sodium"

# Carbohydrates
prob += pulp.lpSum([carbohydrates[food] * food_vars[food] for food in carbohydrates]) >= min_intake['Carbohydrates g'], "Min Carbohydrates"
prob += pulp.lpSum([carbohydrates[food] * food_vars[food] for food in carbohydrates]) <= max_intake['Carbohydrates g'], "Max Carbohydrates"

# Dietary Fiber
prob += pulp.lpSum([fiber[food] * food_vars[food] for food in fiber]) >= min_intake['Dietary_Fiber g'], "Min Fiber"
prob += pulp.lpSum([fiber[food] * food_vars[food] for food in fiber]) <= max_intake['Dietary_Fiber g'], "Max Fiber"


### Solve the Model and Display Results

In [ ]:
# Solve the problem
prob.solve()

# Display the status of the solution
print("Status:", pulp.LpStatus[prob.status])

# Print each food item and its optimal quantity in the diet
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

# Display the minimum cost of the diet
print("Minimum Cost:", pulp.value(prob.objective))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yyy/miniconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/41/k32gp2kd2z53cp_d0tlm52hc0000gn/T/937f946b11f14a598b29413595b6a73c-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/41/k32gp2kd2z53cp_d0tlm52hc0000gn/T/937f946b11f14a598b29413595b6a73c-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 790 RHS
At line 803 BOUNDS
At line 804 ENDATA
Problem MODEL has 12 rows, 64 columns and 708 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 12 (0) rows, 64 (0) columns and 708 (0) elements
0  Obj 0 Primal inf 195.57344 (6)
6  Obj 2.1771457
Optimal - objective value 2.1771457
Optimal objective 2.177145703 - 6 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds)

### Conclusion
The diet problem solution suggests a low-cost combination of raw carrots, raw celery, frozen broccoli, and air-popped popcorn that satisfies all nutritional requirements. This outcome might not be the most appetizing but is cost-effective based on the provided constraints.